In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

from function import log
import config


import pandas_gbq
import google
from google.oauth2 import service_account

In [102]:
# access garmin and authentificate

s=Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=s)
driver.get("https://sso.garmin.com/sso/login?service=https%3A%2F%2Fconnect.garmin.com%2Fpost-auth%2Flogin&webhost=olaxpw-connect04&source=https%3A%2F%2Fconnect.garmin.com%2Fen-US%2Fsignin&redirectAfterAccountLoginUrl=https%3A%2F%2Fconnect.garmin.com%2Fpost-auth%2Flogin&redirectAfterAccountCreationUrl=https%3A%2F%2Fconnect.garmin.com%2Fpost-auth%2Flogin&gauthHost=https%3A%2F%2Fsso.garmin.com%2Fsso&locale=en_US&id=gauth-widget&cssUrl=https%3A%2F%2Fstatic.garmincdn.com%2Fcom.garmin.connect%2Fui%2Fcss%2Fgauth-custom-v1.1-min.css&clientId=GarminConnect&rememberMeShown=true&rememberMeChecked=false&createAccountShown=true&openCreateAccount=false&usernameShown=false&displayNameShown=false&consumeServiceTicket=false&initialFocus=true&embedWidget=false&generateExtraServiceTicket=false#")

#wait until sign-in fields are visible
#wait = WebDriverWait(driver, 60)
#wait.until(EC.frame_to_be_available_and_switch_to_it(("id","gauth-widget-frame-gauth-widget")))
#wait.until(EC.presence_of_element_located(("id","username")))

#write login info to fields, then submit
print("Signing in to connect.garmin.com")
element = driver.find_element_by_id("username")
element.send_keys(config.USER_EMAIL)
element = driver.find_element_by_id("password")
element.send_keys(config.USER_PASS)
element.send_keys(Keys.RETURN)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/florianniclause/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


Signing in to connect.garmin.com


<ipython-input-102-dce7bb299659>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("username")
<ipython-input-102-dce7bb299659>:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("password")


In [3]:
from datetime import datetime

datelist = pd.date_range(start='2022-06-01',end=datetime.today().strftime('%Y-%m-%d'))

pd_date = pd.DataFrame(datelist,columns=['date'])

pd_date

,date
0,2022-06-01
1,2022-06-02
2,2022-06-03
3,2022-06-04
4,2022-06-05
5,2022-06-06
6,2022-06-07
7,2022-06-08
8,2022-06-09
9,2022-06-10


In [4]:
pd_date['score_sleep']='No data'
pd_date['Duree_sommeil']='No data'
pd_date['coucher']='No data'
pd_date['lever']='No data'
pd_date['FC_repos']='No data'
pd_date['FC_max_day']='No data'
pd_date['stress']='No data'
pd_date['training_status']='No data'
pd_date['training_comment']='No data'
pd_date['VO2_max_run']='No data'
pd_date['training_load']='No data'

In [17]:
driver.find_element_by_xpath("//div[contains(@class, 'TrainingStatus')]").text.split('\n')

<ipython-input-17-750456b6d773>:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//div[contains(@class, 'TrainingStatus')]").text.split('\n')


['Productif',
 "Manque de charge d'aérobie élevée",
 'VO2 Max ',
 '57',
 'Charge ',
 '1,469',
 'Altitude ',
 '0 m',
 'Chaleur ',
 '34%']

In [20]:
for i in range(0,datelist.shape[0]):
    driver.get(f"https://connect.garmin.com/modern/sleep/{datelist[i].strftime('%Y-%m-%d')}/score")
    time.sleep(5)
    try :
        print(f"{datelist[i].strftime('%Y-%m-%d')}")
        print(driver.find_element_by_xpath('//div[contains(@class, "SleepScoreSummary_sleepScoreNumber")]').text)
        pd_date.at[i,'score_sleep'] = driver.find_element_by_xpath('//div[contains(@class, "SleepScoreSummary_sleepScoreNumber")]').text
    except:
        pass
    driver.get(f"https://connect.garmin.com/modern/sleep/{datelist[i].strftime('%Y-%m-%d')}")
    time.sleep(5)
    try:
        sleep_hour=driver.find_element_by_xpath("//div[contains(@class, 'SleepTimeEditor')]").text.split('\n')
        pd_date.at[i,'coucher']=sleep_hour[0]
        pd_date.at[i,'lever']=sleep_hour[2]
    except:
        pass
    try:
        duree=driver.find_element_by_xpath("//div[contains(@class, 'sleepGaugeContainer')]").text.replace('h','').replace('m','').split('\n')[0].split(' ')
        pd_date.at[i,'Duree_sommeil']=int(duree[0])*60+int(duree[1])
    except:
        pass
    driver.get(f"https://connect.garmin.com/modern/daily-summary/{datelist[i].strftime('%Y-%m-%d')}/heartRate")
    time.sleep(10)
    try:
        hr=driver.find_element_by_xpath("//div[contains(@class, 'HeartRateCardMain')]").text.split('\n')
        pd_date.at[i,'FC_repos']=hr[2].replace(' bpm','')
        pd_date.at[i,'FC_max_day']=hr[4].replace(' bpm','')
    except:
        pass
    try:
        pd_date.at[i,'stress']=driver.find_element_by_xpath("//div[contains(@class, 'StressCardMain')]").text.split('\n')[0]
    except:

        pass
    try:
        training_status=driver.find_element_by_xpath("//div[contains(@class, 'TrainingStatus')]").text.split('\n')
        pd_date.at[i,'training_status']=training_status[0]
        pd_date.at[i,'training_comment']=training_status[1]
        pd_date.at[i,'VO2_max_run']=training_status[3]
        pd_date.at[i,'training_load']=training_status[5]
    except:
        pass
    pd_date.to_csv('health.csv', index=False)



2022-06-01


<ipython-input-20-06d292a80ea0>:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  print(driver.find_element_by_xpath('//div[contains(@class, "SleepScoreSummary_sleepScoreNumber")]').text)


36


<ipython-input-20-06d292a80ea0>:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  pd_date.at[i,'score_sleep'] = driver.find_element_by_xpath('//div[contains(@class, "SleepScoreSummary_sleepScoreNumber")]').text
<ipython-input-20-06d292a80ea0>:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  sleep_hour=driver.find_element_by_xpath("//div[contains(@class, 'SleepTimeEditor')]").text.split('\n')
<ipython-input-20-06d292a80ea0>:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  duree=driver.find_element_by_xpath("//div[contains(@class, 'sleepGaugeContainer')]").text.replace('h','').replace('m','').split('\n')[0].split(' ')
<ipython-input-20-06d292a80ea0>:26: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  hr=driver

2022-06-02
51
2022-06-03
67
2022-06-04
49
2022-06-05
72
2022-06-06
76
2022-06-07
53
2022-06-08
32
2022-06-09
2022-06-10
50
2022-06-11
60
2022-06-12
29
2022-06-13
85
2022-06-14
2022-06-15
38
2022-06-16


In [ ]:
pd_date

In [ ]:
pd_date.to_csv('health.csv', index=False)